# 🔮 CSIRO Image2Biomass - 推論ノートブック (Kaggle)

## 📋 このノートブックの機能
- ✅ `config.yaml` と `best_fold*.pth` を読み込み
- ✅ test 画像を推論（画像入力のみ）
- ✅ （学習が log1p の場合）expm1 で元スケールへ戻す
- ✅ 提出形式（sample_id, target）の `submission.csv` を生成

## 🚀 使い方
1. Kaggle Notebook に以下2つを **Add Data** する
   - 競技データセット（test.csv / sample_submission.csv / test画像）
   - 学習済み成果物データセット（config.yaml / best_fold0.pth など）

2. このノートブックを実行

## 📝 注意事項
- Kaggle は internet off のことが多いので、推論時は `pretrained=False` に固定
- 予測値は mass（g）なので、負値は 0 にクリップ

## 📦 ライブラリのインポート

In [1]:
# ========================
# Import libraries
# ========================
import os
import re
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import polars as pl
from PIL import Image
from typing import Union, Any

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import yaml
from tqdm.auto import tqdm

print("✅ ライブラリのインポート完了")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

✅ ライブラリのインポート完了


/usr/local/lib/python3.12/dist-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


## 🔧 ユーティリティ関数

### 🌱 Seed固定関数などを実装

In [2]:
# ========== 汎用表示・ログ系 ==========
def sep(word: str, num: int = 80):
    print("=" * num)
    print(word)
    print("=" * num)

def show_df(df: Union[pd.DataFrame, pl.DataFrame], num: int = 3, show_tail: bool = False):
    """データフレームの概要を表示"""
    print(df.shape)
    display(df.head(num))
    if show_tail:
        display(df.tail(num))


# ========== ファイル操作 ==========
def glob_walk(root: Union[str, Path], pattern: str) -> list[Path]:
    """指定パターンにマッチするファイルパスを取得"""
    path = Path(root)
    return sorted(list(path.glob(pattern)))

# ========== seed固定 ==========
def seed_everything(seed: int = 1129) -> None:
    """
    再現性のためのseed固定。
    推論では必須ではないが、念のため設定。
    Args:
        seed (int): 乱数シード値
    """
    import random

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    print(f"🌱 Seed固定完了: {seed}")

### 📄 YAML読み込み関数

In [3]:
def _load_yaml(path: Path) -> Dict:
    """
    YAMLファイルを読み込む。
    
    Args:
        path (Path): YAMLファイルのパス
    
    Returns:
        Dict: 設定情報の辞書
    """
    with open(path, "r") as f:
        cfg = yaml.safe_load(f)

    return cfg

### 🔍 自動検出関数

In [4]:
def _pick_state_dict(ckpt: Dict) -> Dict:
    """
    checkpoint dict から model state_dict を取り出す。
    
    Args:
        ckpt (Dict): チェックポイントの辞書
    
    Returns:
        Dict: モデルのstate_dict
    
    Raises:
        KeyError: state_dictが見つからない場合
    """
    for key in ["model_state_dict", "state_dict", "model"]:
        if key in ckpt and isinstance(ckpt[key], dict):
            return ckpt[key]

    # 直に state_dict っぽい場合
    if all(isinstance(k, str) for k in ckpt.keys()):
        return ckpt
    
    raise KeyError("❌ state_dict が見つかりません。checkpoint の保存形式を確認してください。")

## 🧠 モデル定義

In [5]:
class ConvNeXtRegressor(nn.Module):
    """
    ConvNeXt backbone + Linear head の回帰モデル（画像のみ入力）。
    
    Args:
        backbone (str): timm のモデル名（例: "convnext_base"）
        pretrained (bool): 推論では False 推奨（internet off 対策）
        num_targets (int): 出力ターゲット数（本コンペは 5）
        in_chans (int): 入力チャンネル（通常3）
        drop_rate (float): timm側のdropout rate
        drop_path_rate (float): timm側のdrop path rate
        head_dropout (float): head手前のdropout
    """

    def __init__(
        self,
        backbone: str = "convnext_base",
        pretrained: bool = False,
        num_targets: int = 5,
        in_chans: int = 3,
        drop_rate: float = 0.0,
        drop_path_rate: float = 0.0,
        head_dropout: float = 0.0,
    ) -> None:
        super().__init__()

        self.backbone = timm.create_model(
            backbone,
            pretrained=pretrained,     # ✅ KaggleではFalse推奨（ckptロード前提）
            num_classes=0,             # feature extractor
            global_pool="avg",
            in_chans=in_chans,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
        )
        feat_dim = self.backbone.num_features

        self.head_dropout = (
            nn.Dropout(head_dropout) if head_dropout and head_dropout > 0 
            else nn.Identity()
        )
        self.head = nn.Linear(feat_dim, num_targets)
        
        print(f"🧠 モデル構築完了: {backbone}")
        print(f"   - 特徴量次元: {feat_dim}")
        print(f"   - 出力ターゲット数: {num_targets}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        順伝播。
        
        Args:
            x (torch.Tensor): 入力画像 (B, C, H, W)
        
        Returns:
            torch.Tensor: 予測値 (B, num_targets)
        """
        feat = self.backbone(x)          # (B, feat_dim)
        feat = self.head_dropout(feat)
        out = self.head(feat)            # (B, num_targets)
        return out

## 🖼️ データセット & Transform

### 🔄 Transform定義

In [6]:
def build_infer_transform(
    img_size: int, 
    mean: List[float], 
    std: List[float]
) -> A.Compose:
    """
    推論用のtransform（最小構成）。
    
    Args:
        img_size (int): リサイズ後の画像サイズ
        mean (List[float]): 正規化の平均値
        std (List[float]): 正規化の標準偏差
    
    Returns:
        A.Compose: Albumentationsのtransform
    """
    return A.Compose(
        [
            A.Resize(img_size, img_size),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ]
    )

### 📦 Dataset定義

In [7]:
class TestImageDataset(Dataset):
    """
    test画像（unique image_path）用Dataset。
    
    Args:
        df_unique (pd.DataFrame): ユニークな画像パスを含むDataFrame
        data_root (Path): データのルートディレクトリ
        transform (A.Compose): 適用するtransform
    """

    def __init__(
        self, 
        df_unique: pd.DataFrame, 
        data_root: Path, 
        transform: A.Compose
    ):
        self.df = df_unique.reset_index(drop=True)
        self.data_root = data_root
        self.transform = transform

        self.image_paths = self.df["image_path"].astype(str).values
        
        print(f"📦 データセット作成完了: {len(self)} 画像")

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict:
        """
        インデックスに対応する画像を取得。
        
        Args:
            idx (int): インデックス
        
        Returns:
            Dict: {"image": Tensor, "image_path": str}
        """
        rel_path = self.image_paths[idx]
        img_path = self.data_root / rel_path

        img = Image.open(img_path).convert("RGB")
        img = np.array(img)

        x = self.transform(image=img)["image"]  # torch.Tensor(C,H,W)
        return {"image": x, "image_path": rel_path}

## 🚀 推論コア関数

### 🔮 単一モデル推論

In [8]:
@torch.inference_mode()
def predict_one_ckpt(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device,
    use_amp: bool = True,
) -> Tuple[List[str], np.ndarray]:
    """
    1つのcheckpointで推論を実行。
    
    Args:
        model (nn.Module): 推論に使用するモデル
        loader (DataLoader): データローダー
        device (torch.device): 計算デバイス
        use_amp (bool): AMP（自動混合精度）を使用するか
    
    Returns:
        Tuple[List[str], np.ndarray]: 
            - paths: image_path のリスト
            - preds: (N, K) の予測（モデル出力そのまま：log1p学習なら log1p空間）
    """
    model.eval()

    paths_all: List[str] = []
    preds_all: List[np.ndarray] = []

    pbar = tqdm(loader, desc="🔮 推論中", leave=False)
    for batch in pbar:
        x = batch["image"].to(device, non_blocking=True)

        with autocast(device_type="cuda", enabled=(use_amp and device.type == "cuda")):
            pred = model(x)  # (B, K)

        pred = pred.float().cpu().numpy()
        preds_all.append(pred)
        paths_all.extend(list(batch["image_path"]))

    preds = np.concatenate(preds_all, axis=0)
    return paths_all, preds

### 🎯 アンサンブル推論

In [9]:
def ensemble_predict(
    cfg: Dict,
    ckpt_paths: List[Path],
    unique_test_df: pd.DataFrame,
    data_root: Path,
    device: torch.device,
    batch_size: int = 64,
    num_workers: int = 2,
    use_amp: bool = True,
) -> pd.DataFrame:
    """
    複数checkpointの平均アンサンブル推論（1個なら単体推論）。
    
    Args:
        cfg (Dict): 設定情報の辞書
        ckpt_paths (List[Path]): チェックポイントファイルのパスリスト
        unique_test_df (pd.DataFrame): ユニークな画像パスを含むDataFrame
        data_root (Path): データのルートディレクトリ
        device (torch.device): 計算デバイス
        batch_size (int): バッチサイズ
        num_workers (int): DataLoaderのワーカー数
        use_amp (bool): AMP（自動混合精度）を使用するか
    
    Returns:
        pd.DataFrame: 予測結果を含むDataFrame（image_pathごと）
    """
    sep(f"🎯 アンサンブル推論開始")
    
    # 設定の取得
    img_size = int(cfg.get("img_size", 224))
    norm = cfg.get("normalize", {})
    mean = norm.get("mean", [0.485, 0.456, 0.406])
    std = norm.get("std", [0.229, 0.224, 0.225])

    target_cols = list(cfg["target_cols"])
    use_log1p_target = bool(cfg.get("use_log1p_target", True))
    
    print(f"📐 画像サイズ: {img_size}x{img_size}")
    print(f"📊 ターゲット: {target_cols}")
    print(f"📈 Log1p変換: {use_log1p_target}")

    # transform / loader
    tfm = build_infer_transform(img_size, mean, std)
    ds = TestImageDataset(unique_test_df, data_root=data_root, transform=tfm)
    loader = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False,
    )

    # 予測の蓄積
    pred_sum = None
    paths_ref = None

    print(f"\n🔄 {len(ckpt_paths)}個のモデルで推論中...")
    
    for i, ckpt_path in enumerate(ckpt_paths):
        print(f"\n📦 モデル {i+1}/{len(ckpt_paths)}: {ckpt_path.name}")
        
        # 🧠 モデル構築（pretrainedは必ずFalse）
        model_cfg = cfg.get("model", {})
        model = ConvNeXtRegressor(
            backbone=str(model_cfg.get("backbone", "convnext_base")),
            pretrained=False,  # ✅ Kaggle対応
            num_targets=len(target_cols),
            in_chans=int(model_cfg.get("in_chans", 3)),
            drop_rate=float(model_cfg.get("drop_rate", 0.0)),
            drop_path_rate=float(model_cfg.get("drop_path_rate", 0.0)),
            head_dropout=float(model_cfg.get("head_dropout", 0.0)),
        ).to(device)

        # チェックポイント読み込み
        ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=False)
        state_dict = _pick_state_dict(ckpt)
        missing, unexpected = model.load_state_dict(state_dict, strict=False)
        
        if len(missing) > 0:
            print(f"   ⚠️  missing keys: {len(missing)}")
        if len(unexpected) > 0:
            print(f"   ⚠️  unexpected keys: {len(unexpected)}")

        # 推論実行
        paths, pred = predict_one_ckpt(model, loader, device=device, use_amp=use_amp)

        if paths_ref is None:
            paths_ref = paths
        else:
            assert paths_ref == paths, "❌ 画像の順序が一致しません"

        if pred_sum is None:
            pred_sum = pred
        else:
            pred_sum += pred

        # メモリ解放
        del model
        torch.cuda.empty_cache()

    # 平均を計算
    preds = pred_sum / float(len(ckpt_paths))
    print(f"\n✅ アンサンブル平均計算完了")

    # log->raw（学習がlog1pの場合）
    if use_log1p_target:
        print(f"🔄 Log1p逆変換を適用中...")
        preds = np.expm1(np.clip(preds, -20.0, 20.0))

    # massは非負なので0未満をクリップ
    preds = np.clip(preds, 0.0, None)
    print(f"✂️  負値を0にクリップ")

    # 予測結果のDataFrame作成（image_pathごと）
    pred_df = pd.DataFrame({"image_path": paths_ref})
    for j, col in enumerate(target_cols):
        pred_df[col] = preds[:, j]
    
    return pred_df

## 📋 提出ファイル作成関数

In [10]:
def make_submission(
    artifact_root: str,
    comp_root: Optional[str] = None,
    output_csv: str = "submission.csv",
    fold: str = "0",
    batch_size: int = 64,
    num_workers: int = 2,
    use_amp: bool = True,
    device: Optional[str] = None,
) -> pd.DataFrame:
    """
    submission.csv を作成するメイン関数。
    
    Notebookからこの関数を呼ぶだけで提出ファイルが作成されます。
    
    Args:
        artifact_root (str): Kaggle Datasetのroot（config.yaml / best_fold0.pth を含む）
        comp_root (Optional[str]): competition dataset root（Noneなら自動検出）
        output_csv (str): 出力ファイル名
        fold (str): "0" / "1" / ... / "ALL"（複数foldでアンサンブル）
        batch_size (int): 推論バッチサイズ
        num_workers (int): DataLoaderのワーカー数
        use_amp (bool): AMP推論するか（GPUならTrue推奨）
        device (Optional[str]): "cuda" / "cpu"（Noneなら自動）
    
    Returns:
        pd.DataFrame: 提出用DataFrame（columns=["sample_id", "target"]）
    """
    print()
    sep("📝 提出ファイル作成開始")   
    artifact_root = Path(artifact_root)

    # ✅ デバイス設定
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(device)
    print(f"💻 使用デバイス: {device}")

    # ✅ 競技ディレクトリ
    comp_dir = Path("/kaggle/input/csiro-biomass")
    test_csv = comp_dir / "test.csv"
    sub_csv  = comp_dir / "sample_submission.csv"
    if not test_csv.exists():
        raise FileNotFoundError(f"❌ test.csv が見つかりません: {test_csv}")
    if not sub_csv.exists():
        raise FileNotFoundError(f"❌ sample_submission.csv が見つかりません: {sub_csv}")

    # ✅ 成果物: config + checkpoint
    cfg_path   = artifact_root / "yaml" / "config.yaml" 
    ckpt_paths = glob_walk(artifact_root / "model", "*pth")
    cfg = _load_yaml(cfg_path)
    
    target_cols = list(cfg["target_cols"])
    print(f"\n📊 設定情報:")
    print(f"   - ターゲット: {target_cols}")
    print(f"   - Log1p変換: {cfg.get('use_log1p_target', True)}")

    # ✅ テストデータ読み込み
    print(f"\n📂 テストデータ読み込み中...")
    test_df = pd.read_csv(test_csv)
    
    # 必須カラムのチェック
    required_cols = {"sample_id", "image_path", "target_name"}
    if not required_cols.issubset(set(test_df.columns)):
        raise KeyError(
            f"❌ test.csv に必須カラムがありません。\n"
            f"   必要: {required_cols}\n"
            f"   実際: {test_df.columns.tolist()}"
        )    
    print(f"✅ テストデータ読み込み完了: {len(test_df)} 行")

    # ユニークな画像パスを取得
    unique_test_df = test_df[["image_path"]].drop_duplicates().reset_index(drop=True)
    print(f"🖼️  ユニークな画像数: {len(unique_test_df)}")
    # ✅ 推論実行
    pred_df = ensemble_predict(
        cfg=cfg,
        ckpt_paths=ckpt_paths,
        unique_test_df=unique_test_df,
        data_root=comp_dir,   # test/xxx.jpg が comp_dir 配下にある想定
        device=device,
        batch_size=batch_size,
        num_workers=num_workers,
        use_amp=use_amp,
    )

    # ✅ マージ & 提出ファイル作成
    print(f"\n🔗 予測結果をマージ中...")
    merged = test_df.merge(pred_df, on="image_path", how="left")
    
    # 欠損値チェック
    if merged[target_cols].isna().any().any():
        n_miss = int(merged[target_cols].isna().any(axis=1).sum())
        raise ValueError(f"❌ マージ後に欠損値が発生: {n_miss} 行")

    # target_nameに対応する列を選択
    print(f"🎯 target_name に対応する値を抽出中...")
    out = np.zeros(len(merged), dtype=np.float32)
    for col in target_cols:
        m = (merged["target_name"] == col)
        out[m.values] = merged.loc[m, col].values.astype(np.float32)

    # 提出用DataFrame作成
    submission_df = pd.DataFrame({
        "sample_id": merged["sample_id"].values, 
        "target": out
    })
    
    # CSV保存
    submission_df.to_csv(output_csv, index=False)

    print(); sep(f"✅ 提出ファイル保存完了: {output_csv}")
    print(f"\n📄 提出ファイルプレビュー:")
    show_df(submission_df, 5, False)
    
    return submission_df

## 🚀 実行例

### 基本的な使い方

In [11]:
# Seed固定（再現性のため）
seed_everything(1129)

# 提出ファイル作成
# artifact_root には学習済みモデルとconfigが含まれるデータセットのパスを指定
submission = make_submission(
    artifact_root="/kaggle/input/csiro-artifacts",  # ← 要変更
    output_csv="submission.csv",
    fold="0",           # 使用するfold（"0", "1", ... or "ALL"）
    batch_size=64,      # バッチサイズ
    num_workers=2,      # DataLoaderのワーカー数
    use_amp=True,       # AMP使用（GPU推奨）
)

# 結果確認
print("\n🎉 処理完了!")
submission.head()

🌱 Seed固定完了: 1129

📝 提出ファイル作成開始
💻 使用デバイス: cuda

📊 設定情報:
   - ターゲット: ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
   - Log1p変換: True

📂 テストデータ読み込み中...
✅ テストデータ読み込み完了: 5 行
🖼️  ユニークな画像数: 1
🎯 アンサンブル推論開始
📐 画像サイズ: 224x224
📊 ターゲット: ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
📈 Log1p変換: True
📦 データセット作成完了: 1 画像

🔄 1個のモデルで推論中...

📦 モデル 1/1: best_fold0.pth
🧠 モデル構築完了: convnext_base
   - 特徴量次元: 1024
   - 出力ターゲット数: 5
   ⚠️  missing keys: 342
   ⚠️  unexpected keys: 342


🔮 推論中:   0%|          | 0/1 [00:00<?, ?it/s]


✅ アンサンブル平均計算完了
🔄 Log1p逆変換を適用中...
✂️  負値を0にクリップ

🔗 予測結果をマージ中...
🎯 target_name に対応する値を抽出中...

✅ 提出ファイル保存完了: submission.csv

📄 提出ファイルプレビュー:
(5, 2)


,sample_id,target
0,ID1001187975__Dry_Clover_g,0.000000
1,ID1001187975__Dry_Dead_g,0.250154
2,ID1001187975__Dry_Green_g,16.725424
3,ID1001187975__Dry_Total_g,0.000000
4,ID1001187975__GDM_g,0.391756



🎉 処理完了!


,sample_id,target
0,ID1001187975__Dry_Clover_g,0.000000
1,ID1001187975__Dry_Dead_g,0.250154
2,ID1001187975__Dry_Green_g,16.725424
3,ID1001187975__Dry_Total_g,0.000000
4,ID1001187975__GDM_g,0.391756


### 複数Foldのアンサンブル推論

In [12]:
# 全てのfoldを使ってアンサンブル推論
submission_ensemble = make_submission(
    artifact_root="/kaggle/input/csiro-artifacts",  # ← 要変更
    output_csv="submission.csv",
    fold="ALL",         # 全foldを使用
    batch_size=64,
    num_workers=2,
    use_amp=True,
)

submission_ensemble.head()


📝 提出ファイル作成開始
💻 使用デバイス: cuda

📊 設定情報:
   - ターゲット: ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
   - Log1p変換: True

📂 テストデータ読み込み中...
✅ テストデータ読み込み完了: 5 行
🖼️  ユニークな画像数: 1
🎯 アンサンブル推論開始
📐 画像サイズ: 224x224
📊 ターゲット: ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
📈 Log1p変換: True
📦 データセット作成完了: 1 画像

🔄 1個のモデルで推論中...

📦 モデル 1/1: best_fold0.pth


Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 259, in _feed
    reader_close()
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 291, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times


🧠 モデル構築完了: convnext_base
   - 特徴量次元: 1024
   - 出力ターゲット数: 5
   ⚠️  missing keys: 342
   ⚠️  unexpected keys: 342


🔮 推論中:   0%|          | 0/1 [00:00<?, ?it/s]


✅ アンサンブル平均計算完了
🔄 Log1p逆変換を適用中...
✂️  負値を0にクリップ

🔗 予測結果をマージ中...
🎯 target_name に対応する値を抽出中...

✅ 提出ファイル保存完了: submission.csv

📄 提出ファイルプレビュー:
(5, 2)


,sample_id,target
0,ID1001187975__Dry_Clover_g,0.000000
1,ID1001187975__Dry_Dead_g,2.231194
2,ID1001187975__Dry_Green_g,1.899253
3,ID1001187975__Dry_Total_g,0.000000
4,ID1001187975__GDM_g,1.939163


,sample_id,target
0,ID1001187975__Dry_Clover_g,0.000000
1,ID1001187975__Dry_Dead_g,2.231194
2,ID1001187975__Dry_Green_g,1.899253
3,ID1001187975__Dry_Total_g,0.000000
4,ID1001187975__GDM_g,1.939163


## 📝 最終チェック

In [13]:
# 提出ファイルの形式チェック
print("📋 提出ファイルチェック:")
print(f"   - 行数: {len(submission)}")
print(f"   - カラム: {submission.columns.tolist()}")
print(f"   - 欠損値: {submission.isna().sum().sum()}")
print(f"   - 負値の数: {(submission['target'] < 0).sum()}")
print(f"   - target の範囲: [{submission['target'].min():.2f}, {submission['target'].max():.2f}]")

📋 提出ファイルチェック:
   - 行数: 5
   - カラム: ['sample_id', 'target']
   - 欠損値: 0
   - 負値の数: 0
   - target の範囲: [0.00, 16.73]


## 🎉 完了!

`submission.csv` が作成されました。Kaggleに提出してください!

### 📊 次のステップ
- 提出結果を確認
- パラメータの調整（batch_size, use_amp等）
- 複数foldのアンサンブルを試す
- 後処理の追加（必要に応じて）